In [1]:
import pandas as pd
import json
import requests

한번에 최대 10000개

3일 단위로 계속 불러오기 6개월까지

In [55]:
query="아이폰13"
url = f'https://search-api.joongna.com/v25/search/product'
data = {
    "filter":{
#         "categoryDepth":0,
#         "categorySeq":0,
#         "dateFilterParameter":{
#             "sortEndDate":"2022-04-15",
#             "sortStartDate":"2022-04-13"
#         },
        "productCondition":-1,
        "flawedYn":0,
        "fullPackageYn":0,
        "limitedEditionYn":0,
        "maxPrice":2000000000,
        "minPrice":0,
        "tradeType":0
    },
    "page":0,
    "firstQuantity":10000,
    "productFilter":"ALL",
    "productStates":[
#         0,
#         1,
        99
    ],
    "quantity":10000,
    "searchQuantity":10000,
    "osType":2,
    "searchWord":"아이폰13",
    "sort":"RECENT_SORT",
    "startIndex":0,
#     "searchStartTime":"2022-04-15 14:55:21"
}

In [3]:
data_page={
    "filter":{
        "categoryDepth":0,
        "categorySeq":0,
        "dateFilterParameter":{
            "sortEndDate":"2022-04-15",
            "sortStartDate":"2022-04-14"
        },
        "productCondition":-1,
        "flawedYn":0,
        "fullPackageYn":0,
        "limitedEditionYn":0,
        "maxPrice":2000000000,
        "minPrice":0,
        "tradeType":0
    },
    "page":21,
    "firstQuantity":10,
    "productFilter":"ALL",
    "productStates":[
        0,
        1
    ],
    "quantity":10,
    "searchQuantity":10,
    "osType":2,
    "searchWord":"아이폰13",
    "sort":"RECENT_SORT",
    "startIndex":21,
    "searchStartTime":"2022-04-15 16:42:07"
}

In [4]:
n={
    "filter":{
        "categoryDepth":0,
        "categorySeq":0,
        "dateFilterParameter":{
            "sortEndDate":"2022-04-15",
            "sortStartDate":"2022-04-14"
        },
        "productCondition":-1,
        "flawedYn":0,
        "fullPackageYn":0,
        "limitedEditionYn":0,
        "maxPrice":2000000000,
        "minPrice":0,
        "tradeType":0
    },
    "page":22,
    "firstQuantity":10,
    "productFilter":"ALL",
    "productStates":[
        0,
        1
    ],
    "quantity":10,
    "searchQuantity":10,
    "osType":2,
    "searchWord":"아이폰13",
    "sort":"RECENT_SORT",
    "startIndex":22,
    "searchStartTime":"2022-04-15 16:43:24"
}

In [56]:
#json.dumps
response= requests.post(url,json=(data))
data=response.json()['data']

In [57]:
len(data['items'])

10000

In [9]:
data['items'][0]

{'seq': 0,
 'productPositionNo': 1,
 'highlightedTitle': '<em>아이폰</em><em>13</em> 프로 pro 12 미개봉 자급제 중고폰 매입 삽니다',
 'storeSeq': 0,
 'platformType': 4,
 'storeProfileUrl': 'https://img2.joongna.com/common/profile_naver_img.jpg?impolicy=thumb&size=50',
 'storeNickname': '',
 'userName': '',
 'detailImgUrl': None,
 'mainLocationName': '',
 'locationNames': None,
 'articleUrl': 'https://m.cafe.naver.com/ArticleRead.nhn?clubid=10050146&menuid=424&boardtype=C&page=1&articleid=911021625&referrerAllArticles=false',
 'menuId': 424,
 'articleSeq': 911021625,
 'articleRegDate': '2022-04-15 23:59:50',
 'articleBadgeShow': 0,
 'consignmentType': 0,
 'url': '/cafe/2022/04/16/1650034854148_e309dadb-3a54-4546-9670-bc8ba0523225.jpeg?impolicy=thumb&size=150',
 'productHiddenStatus': 0,
 'price': 1000000,
 'state': 0,
 'title': '아이폰13 프로 pro 12 미개봉 자급제 중고폰 매입 삽니다',
 'subTitle': None,
 'sortDate': '2022-04-15 23:59:50',
 'userSeq': 0,
 'videoUrl': '',
 'viewCount': 0,
 'videoProductYn': None,
 'productType'

In [8]:
data['items'][1000]

{'seq': 0,
 'productPositionNo': 1001,
 'highlightedTitle': '[매입]<em>아이폰</em><em>13</em>,갤럭시S22전기종 자급제,박스폰,중고폰,파손폰매입',
 'storeSeq': 0,
 'platformType': 4,
 'storeProfileUrl': 'https://img2.joongna.com/common/profile_naver_img.jpg?impolicy=thumb&size=50',
 'storeNickname': '',
 'userName': '',
 'detailImgUrl': None,
 'mainLocationName': '',
 'locationNames': None,
 'articleUrl': 'https://m.cafe.naver.com/ArticleRead.nhn?clubid=10050146&menuid=424&boardtype=C&page=1&articleid=910853437&referrerAllArticles=false',
 'menuId': 424,
 'articleSeq': 910853437,
 'articleRegDate': '2022-04-15 08:26:22',
 'articleBadgeShow': 0,
 'consignmentType': 0,
 'url': '/cafe/2022/04/15/1649978815907_fdb32cac-33dd-4074-9135-2d94234faa16.jpeg?impolicy=thumb&size=150',
 'productHiddenStatus': 0,
 'price': 2000000,
 'state': 0,
 'title': '[매입]아이폰13,갤럭시S22전기종 자급제,박스폰,중고폰,파손폰매입',
 'subTitle': None,
 'sortDate': '2022-04-15 08:26:22',
 'userSeq': 0,
 'videoUrl': '',
 'viewCount': 0,
 'videoProductYn': None,
 'pro

In [1]:
articleUrl
articleSeq
articleRegDate
price
title
sortDate
productHiddenStatus # 노상관
state # 0 1 3

NameError: name 'title' is not defined

In [45]:
df=pd.DataFrame(data['items'])
df

,seq,productPositionNo,highlightedTitle,storeSeq,platformType,storeProfileUrl,storeNickname,userName,detailImgUrl,mainLocationName,...,viewCount,videoProductYn,productType,ad,adImpUrl,adClickUrl,adType,isConsignment,jnPayBadgeFlag,intJnPayBadgeFlag
0,0,1,<em>아이폰</em><em>13</em> 프로 맥스 갤럭시S22 울트라 전기종 삽...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
1,0,2,"[매입]<em>아이폰</em><em>13</em>,폴드,플립전기종 자급제,박스폰,중...",0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
2,0,3,<em>아이폰</em><em>13</em>프로 시에라블루 256용량 새상품컨디션 초...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
3,0,4,정직매입 <em>아이폰</em><em>13</em> 프로 맥스 256G 128G 개...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
4,0,5,"[매입]<em>아이폰</em><em>13</em> 프로 맥스 애플전기종,중고폰,자급...",0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,9996,"[매입]<em>아이폰</em><em>13</em>,갤럭시S22전기종 자급제,박스폰,...",0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
9996,0,9997,<em>아이폰</em><em>13</em>프로 128G 블루 리퍼폰 새제품 팝니다.,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
9997,0,9998,164456 미개봉 새제품 <em>아이폰</em><em>13</em> 블루 128G...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
9998,0,9999,<em>아이폰</em><em>13</em>프로 맥스 미니 갤럭시S22 자급제 통신사...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0


In [46]:
df.columns

Index(['seq', 'productPositionNo', 'highlightedTitle', 'storeSeq',
       'platformType', 'storeProfileUrl', 'storeNickname', 'userName',
       'detailImgUrl', 'mainLocationName', 'locationNames', 'articleUrl',
       'menuId', 'articleSeq', 'articleRegDate', 'articleBadgeShow',
       'consignmentType', 'url', 'productHiddenStatus', 'price', 'state',
       'title', 'subTitle', 'sortDate', 'userSeq', 'videoUrl', 'viewCount',
       'videoProductYn', 'productType', 'ad', 'adImpUrl', 'adClickUrl',
       'adType', 'isConsignment', 'jnPayBadgeFlag', 'intJnPayBadgeFlag'],
      dtype='object')

In [47]:
df['state'].unique()

array([0], dtype=int64)

In [48]:
df[df['state'] == 1]['productPositionNo']

Series([], Name: productPositionNo, dtype: int64)

In [49]:
col=['articleUrl',
'articleSeq',
'articleRegDate',
'price',
'title',
'sortDate',
'productHiddenStatus', # 노상관
'state'] # 0 1 3]
df[col]

,articleUrl,articleSeq,articleRegDate,price,title,sortDate,productHiddenStatus,state
0,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,911775213.0,2022-04-19 16:50:09,1550000,아이폰13 프로 맥스 갤럭시S22 울트라 전기종 삽니다. 자급제 통신사 유심기변,2022-04-19 16:50:09,0,0
1,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,911775115.0,2022-04-19 16:49:36,2000000,"[매입]아이폰13,폴드,플립전기종 자급제,박스폰,중고폰 당일매입",2022-04-19 16:49:36,0,0
2,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,911774845.0,2022-04-19 16:48:14,1130000,아이폰13프로 시에라블루 256용량 새상품컨디션 초S급 113만 판매해요,2022-04-19 16:48:14,0,0
3,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,911774527.0,2022-04-19 16:46:30,1300000,정직매입 아이폰13 프로 맥스 256G 128G 개통폰 단순개봉폰 자급제 새폰,2022-04-19 16:46:30,0,0
4,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,911774514.0,2022-04-19 16:46:27,1500000,"[매입]아이폰13 프로 맥스 애플전기종,중고폰,자급제미개봉,박스폰매입",2022-04-19 16:46:27,0,0
...,...,...,...,...,...,...,...,...
9995,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910225575.0,2022-04-12 09:23:22,2000000,"[매입]아이폰13,갤럭시S22전기종 자급제,박스폰,중고폰,파손폰매입",2022-04-12 09:23:22,0,0
9996,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910225479.0,2022-04-12 09:22:48,1120000,아이폰13프로 128G 블루 리퍼폰 새제품 팝니다.,2022-04-12 09:22:48,0,0
9997,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910224575.0,2022-04-12 09:18:22,970000,164456 미개봉 새제품 아이폰13 블루 128GB 풀박스 97만,2022-04-12 09:18:22,0,0
9998,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,910224178.0,2022-04-12 09:16:17,3000,아이폰13프로 맥스 미니 갤럭시S22 자급제 통신사 유심기변 전기종 삽니다.,2022-04-12 09:16:17,0,0


In [50]:
df.drop_duplicates('title')

,seq,productPositionNo,highlightedTitle,storeSeq,platformType,storeProfileUrl,storeNickname,userName,detailImgUrl,mainLocationName,...,viewCount,videoProductYn,productType,ad,adImpUrl,adClickUrl,adType,isConsignment,jnPayBadgeFlag,intJnPayBadgeFlag
0,0,1,<em>아이폰</em><em>13</em> 프로 맥스 갤럭시S22 울트라 전기종 삽...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
1,0,2,"[매입]<em>아이폰</em><em>13</em>,폴드,플립전기종 자급제,박스폰,중...",0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
2,0,3,<em>아이폰</em><em>13</em>프로 시에라블루 256용량 새상품컨디션 초...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
3,0,4,정직매입 <em>아이폰</em><em>13</em> 프로 맥스 256G 128G 개...,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
4,0,5,"[매입]<em>아이폰</em><em>13</em> 프로 맥스 애플전기종,중고폰,자급...",0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,0,9943,<em>아이폰</em><em>13</em>프로 128GB 미개봉 삽니다.,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
9953,0,9954,<em>아이폰</em><em>13</em> 512기가 자급제미개봉상품 판매합니다,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
9965,0,9966,[미개봉] <em>아이폰</em> <em>13</em> 128gb 레드 판매합니다.,0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
9970,0,9971,<em>아이폰</em><em>13</em>프로맥스용 UAG 모나크 케이스(택포),0,4,https://img2.joongna.com/common/profile_naver_...,,,None,,...,0,None,,0,,,,0,False,0
